In [245]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [246]:
# encodeamos todas los features categoricos usando One Hot encoding


In [351]:
def normaliza_df(df, onehotencoder = None):
    print(df.shape)

    # drops -------------------------------------------------
    '''
      removemos la columna de la cual el modelo podria aprender que su existencia
      implica el Closed_won (ya que es un valor que se obtiene a posteriori de haber
      ganado y estaria mal usarlo ya que los modelos entrenadas con ella serian 
      incapaces de poder predecir correctamente)
    '''
    df = df.drop(['Sales_Contract_No'], axis=1)
    print(df.shape)
    
    # irrelevantes
    df = df.drop(['Opportunity_Name'], axis=1)
    print(df.shape)

    # dato cte = NaT
    df = df.drop(['Last_Activity'], axis=1)
    print(df.shape)
    
    # todo: convertir estas en una ventana de tiempo
    df = df.drop(['Planned_Delivery_Start_Date'], axis=1)
    df = df.drop(['Planned_Delivery_End_Date'], axis=1)
    print(df.shape)
    
    # @todo: no estoy seguro si esto aporta
    df = df.drop(['Quote_Expiry_Date'], axis=1)
    print(df.shape)
    
    # @todo: convertirla a una diferencia de dias?
    df = df.drop(['Last_Modified_Date'], axis=1)
    print(df.shape)
    
    # @todo: existe Opportunity_id 
    df = df.drop(['ID'], axis=1)
    print(df.shape)

    # basado en lo charlado con el grupo, ahora subdividimos el df de forma de tener
    # en cuenta los casos cerrados y cobertimos las varaibles categorias en su
    # representacion mas simple (siendo dos casos en Cloased_Won = 1 y Closed:_lost = 0)
    if 'Stage' in df.columns:
        df = df[((df['Stage'] == 'Closed Won') | (df['Stage'] == 'Closed Lost'))]
        df['Stage'] = df['Stage'].apply(lambda x: 1 if x == 'Closed Won' else 0)
    
    print('-----------------')
    print(df.shape)
    
    # hacemos que las variables temporales en las que nos vanos a enfocar sean del tipo correcto
    df['Account_Created_Date'] = pd.to_datetime(df['Account_Created_Date'], format="%m/%d/%Y")
    df['Opportunity_Created_Date'] = pd.to_datetime(df['Opportunity_Created_Date'], format="%m/%d/%Y")
    
    df['Quote_Type'] = df['Quote_Type'].apply(lambda x: 1 if x == 'Binding' else 0)
    
    #------------------------------------------
    df = df.sort_values(by="Opportunity_Created_Date")
    
    df = df.drop(columns = 'Opportunity_Created_Date')
    df = df.drop(columns = 'Account_Created_Date')
    #------------------------------------------
    
    categoric_cols = df.columns[df.dtypes==object].tolist() 
    numeric_cols = df.columns[df.dtypes=='float64'].tolist() 
    numeric_cols_2 = df.columns[df.dtypes=='int64'].tolist() 
    date_cols = df.columns[df.dtypes=='datetime64[ns]'].tolist() 

    if 'Stage' in df.columns:
        onehotencoder = OneHotEncoder(handle_unknown = 'ignore')
        onehotencoder.fit(df[categoric_cols])
    
    cat_rel = 0
    all_col_names = []
    for cat in onehotencoder.categories_:
        for col in cat:
            all_col_names.append(categoric_cols[cat_rel] + '_' + col)
        cat_rel = cat_rel + 1
    
    categorical = pd.DataFrame(onehotencoder.transform(df[categoric_cols]).toarray(), columns=all_col_names)
    
    
    categorical = categorical.reset_index()
    df = df.reset_index()
    
    print('ante ...')
    print(df.shape)
    print(categorical.shape)
    print(df[numeric_cols].shape)
    print(df[numeric_cols_2].shape)

    
    
    frames_to_concat = [categorical, df[numeric_cols], df[numeric_cols_2]]
    dfx = pd.concat(frames_to_concat, axis=1)
    
    print('final...')
    dfx = dfx.reset_index()
    print(dfx.shape)
        
    return dfx, onehotencoder

In [352]:
df_train = pd.read_csv('/usr/src/tp2-1/Train_TP2_Datos_2020-2C.csv')
df_train = df_train.dropna()

In [353]:
df_train.reset_index()
df_train.shape

(13684, 52)

(13684, 52)

In [354]:
df_train.loc[df_train['Opportunity_ID'] == 0]

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A
0,27761,EMEA,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,...,Q2,2016,NaT,10,EUR,5272800.0,EUR,5272800.0,Closed Lost,Prod_Category_A_None


In [355]:
#df_train.info()
df_train.Price.value_counts()

None     12765
Other      585
0.24        27
0.28        20
0.27        18
0.29        17
0.41        17
0.32        15
0.35        14
0.23        13
0.38        12
0.33        12
0.34        11
0.3         11
0.26        11
0.31        10
0.42        10
0.245        8
0.305        7
0.25         7
0.22         6
0.335        6
0.225        6
0.325        6
0.39         5
0.2          5
0.37         4
0.275        4
0.285        4
0.45         4
0.36         4
0.265        3
0.295        3
0.47         3
0.355        3
0.255        3
0.4          2
0.445        2
0.315        2
0.44         2
0.345        2
0.375        2
0.215        2
0.235        2
0.55         1
0.56         1
0.365        1
0.48         1
0.21         1
0.465        1
0.52         1
0.49         1
0.505        1
Name: Price, dtype: int64

In [356]:
[ndf, onehotencoder] = normaliza_df(df_train)

(13684, 52)
(13684, 51)
(13684, 50)
(13684, 49)
(13684, 47)
(13684, 46)
(13684, 45)
(13684, 44)
-----------------
(13625, 44)
ante ...
(13625, 43)
(13625, 2809)
(13625, 4)
(13625, 10)
final...
(13625, 2824)


In [392]:
ndf.shape

(13625, 2824)

In [393]:
ndf.head()

,level_0,index,Region_APAC,Region_Americas,Region_EMEA,Region_Japan,Region_Middle East,Territory_Albania,Territory_Armenia,Territory_Australia,...,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Opportunity_ID,Quote_Type,Delivery_Year,TRF,Stage
0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,12369,0,2018,2,0
1,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,0,12370,0,2016,20,0
2,2,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,1,0,0,12371,0,2016,0,0
3,3,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,1,0,0,12371,0,2016,0,0
4,4,4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,1,0,0,12372,0,2016,3,1


In [335]:
ndf.loc[ndf['Opportunity_ID'] == 0]

,level_0,index,Region_APAC,Region_Americas,Region_EMEA,Region_Japan,Region_Middle East,Territory_Albania,Territory_Armenia,Territory_Australia,...,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Opportunity_ID,Quote_Type,Delivery_Year,TRF,Stage
497,497,497,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,0,0,0,2016,10,0


In [394]:
ndf2 = ndf.groupby('Opportunity_ID').mean()
ndf2 = ndf2.reset_index()

In [395]:
ndf2.shape

(9739, 2824)

In [337]:
X = ndf2.copy()
y = X.pop("Stage")

# X = ndf.fillna(0)
# X = ndf.dropna()

# Label encoding for categoricals
for colname in X.select_dtypes("object"):
    X[colname], _ = X[colname].factorize()

# All discrete features should now have integer dtypes (double-check this before using MI!)
discrete_features = X.dtypes == int


In [306]:
y

0        0
1        0
2        0
3        0
4        1
        ..
13620    1
13621    1
13622    1
13623    1
13624    1
Name: Stage, Length: 13625, dtype: int64

In [338]:
from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y):
    mi_scores = mutual_info_regression(X, y)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y)
#

In [339]:
mi_scores[0:30]

TRF                                      0.132666
Total_Amount                             0.127657
Total_Taxable_Amount                     0.099485
index                                    0.095868
level_0                                  0.095371
ASP_(converted)                          0.091448
ASP                                      0.073872
Product_Type_None                        0.066882
Product_Category_B_None                  0.061704
Price_None                               0.059844
Brand_None                               0.057873
Size_None                                0.048424
Bureaucratic_Code_Bureaucratic_Code_4    0.040351
Bureaucratic_Code_0_Approval             0.040338
Price_Other                              0.039313
Currency_None                            0.036770
Opportunity_Type_Opportunity_Type_19     0.035580
Brand_Other                              0.034220
Account_Name_Account_Name_712            0.033975
Bureaucratic_Code_Bureaucratic_Code_5    0.033350


In [398]:
ndf2.to_csv('/usr/src/tp2-1/210215_tp2_train_feng.csv')

In [399]:
ndf2

,Opportunity_ID,level_0,index,Region_APAC,Region_Americas,Region_EMEA,Region_Japan,Region_Middle East,Territory_Albania,Territory_Armenia,...,Total_Taxable_Amount,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Quote_Type,Delivery_Year,TRF,Stage
0,0,497.0,497.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,5272800.0,1.0,1.0,1.0,1.0,0.0,0.0,2016.0,10.0,0.0
1,1,498.0,498.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,48230.0,0.0,0.0,0.0,0.0,0.0,0.0,2016.0,0.0,1.0
2,2,499.0,499.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,83865.6,0.0,0.0,0.0,0.0,0.0,0.0,2016.0,0.0,1.0
3,3,500.0,500.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,7421881.5,1.0,0.0,1.0,0.0,0.0,0.0,2018.0,14.0,0.0
4,4,501.0,501.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,13357192.5,1.0,0.0,1.0,0.0,0.0,0.0,2018.0,25.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9734,12799,483.0,483.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,401700.0,1.0,1.0,0.0,0.0,0.0,0.0,2016.0,1.0,1.0
9735,12800,481.5,481.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,21332500.0,1.0,0.0,0.0,0.0,0.0,0.0,2016.0,20.0,0.0
9736,12801,479.0,479.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,299715.0,1.0,1.0,1.0,1.0,0.0,0.0,2016.0,0.0,1.0
9737,12802,496.0,496.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,2016.0,4.0,0.0


In [382]:
df_test = pd.read_csv('/usr/src/tp2-1/Test_TP2_Datos_2020-2C.csv')
df_test = df_test.dropna()

In [383]:
[ndft, _] = normaliza_df(df_test, onehotencoder)

(2467, 51)
(2467, 50)
(2467, 49)
(2467, 48)
(2467, 46)
(2467, 45)
(2467, 44)
(2467, 43)
-----------------
(2467, 43)
ante ...
(2467, 42)
(2467, 2809)
(2467, 4)
(2467, 9)
final...
(2467, 2823)


In [396]:
ndft2 = ndft.groupby('Opportunity_ID').mean()
ndft2 = ndft2.reset_index()

In [397]:
ndft2.shape

(1567, 2823)

In [391]:
ndft2.to_csv('/usr/src/tp2-1/210215_tp2_test_feng.csv')